<div style="text-align:center">
<h1> CSCI 3155 Principles of Programming Languages </h1>
<h2> Spring 2025</h2>
</div>

### Today's Lecture
+ Inductive Definitions
  * A grammar for arithmetic expressions
  * Abstract Syntax Trees (ASTs)
+ Operations on Inductive Definitions
  * `minusOne` function on natural numbers using visitor pattern
  * `minusOne` using pattern matching
  * `plusNum`
  * Operations on lists: `listLen`, `isAscending`, `concat`, `reverse`.
  * Operations on Arithematic ASTs
  

### Arithmetic Expression Grammar

Let us now examine a grammar for arithmetic expressions involving operators like `+`, `*`, `/` and even functions like log, exp, sine and cosine.

$$\begin{array}{rcc}
\textbf{Expr} & \rightarrow & Const(\textbf{Double}) \\
& |  & Ident(\textbf{Identifier}) \\
& | & Plus( \textbf{Expr}, \textbf{Expr}) \\
& | & Minus( \textbf{Expr}, \textbf{Expr}) \\
& | & Mult(\textbf{Expr}, \textbf{Expr}) \\
& | & Div(\textbf{Expr}, \textbf{Expr}) \\
& | & Log(\textbf{Expr}) \\
& | & Exp(\textbf{Expr}) \\
& | & Sine(\textbf{Expr}) \\
& | & Cosine(\textbf{Expr}) \\\\
\textbf{Double} & \rightarrow & \cdots\ |\  -2\ |\ -1\ |\ 0\ |\ 1\ |\ 2\ |\ \cdots \\
\textbf{Identifier} & \rightarrow & [a-z\ A-Z][a-z\ A-Z\ 0-9\ \_]*
\end{array}$$

We will clarify a few things first. The non terminal **Identifier** stands for a string that
represents a variable name like `x`, `y`, `velocity`, `s_29109_12xyZ` and so on. We wrote a regular expression as a stand in for all possible legal variable names that can appear.

The start symbol is **Expr**. 

Let us look at some examples of expressions that can be created this way.

1.  $ x^2 + 5$ is expressed as  Plus(Mult( Ident("x"),  Ident("x")), Const(5))
2.  $ \sin(x) + \cos(x)$ is expressed as Plus(Sine(Ident("x")), Cosine(Ident("x")))
3.  $e^{e^{xy}}$ is expressed as Exp(Exp(Mult( Ident("x"),  Ident("y") ) ))

You can guess from the context what the  symbols *Plus, Minus, Mult, Div, Log,
Exp, Sine* and *Cosine* should mean. The symbols *Const* and *Ident* seem superfluous here. After all, 
if there is a string "x" it can only refer to an identifier and if there is a constant 10, it can only
be an integer. However, for scala (or any other programming language) "x" is a string and 10 is an integer.
We need a means to tell the scala interpreter to *promote* a number 10 to an expression 10 or a string "x_25" to 
the expression "x_25". The easiest way to achieve this is to wrap these inside a symbol such as *Const* or
*Ident* that makes it easy for
the scala interpreter to undertand what is going on.

* Arithmetic  expression: `x^2 + 2.0`
* In my language: `Plus(Mult(Ident("x"), Ident("x")), Const(2.0))`

In [2]:
sealed trait Expr

case class Const(x:Double) extends Expr
case class Ident(v:String) extends Expr
case class Plus(e1:Expr,e2:Expr) extends Expr
case class Mult(e1:Expr,e2:Expr) extends Expr
case class Minus(e1:Expr,e2:Expr) extends Expr
case class Sine(e:Expr) extends Expr

defined trait Expr
defined class Const
defined class Ident
defined class Plus
defined class Mult
defined class Minus
defined class Sine

In [5]:
val e:Expr = Plus(Mult(Ident("x"),Ident("x")), Const(2))//x^2 + 2

e: Expr = Plus(
  e1 = Mult(e1 = Ident(v = "x"), e2 = Ident(v = "x")),
  e2 = Const(x = 2.0)
)

In [8]:
//"Visitor pattern"
sealed trait Num {
    def minusOne():Num
}

case object Z extends Num {
    def minusOne():Num = {
        throw (new IllegalArgumentException("You can't subtract from zero!"))
    }
}
case class Succ(n:Num) extends Num {
    def minusOne():Num = {
        n
    }
}

defined trait Num
defined object Z
defined class Succ

In [9]:
val n1:Num = Succ(Succ(Z)) //two
val n2:Num = n1.minusOne // Succ(Z)

n1: Num = Succ(n = Succ(n = Z))
n2: Num = Succ(n = Z)

In [17]:
//"pattern matching"
def altMinusOne(n:Num) : Num = {
    n match {
        case Succ(n1) => n1
        case _ => throw (new IllegalArgumentException("You can't subtract from zero!"))
    }
}

defined function altMinusOne

In [19]:
altMinusOne(n1)

res18: Num = Succ(n = Z)

In [20]:
sealed trait NumList
case object Nil extends NumList
case class Cons(hd:Num, tl:NumList) extends NumList

defined trait NumList
defined object Nil
defined class Cons

In [21]:
val l1: NumList = Cons(n1, Cons(n2, Nil)) //equivalent to scala list `List(2,1)`

l1: NumList = Cons(
  hd = Succ(n = Succ(n = Z)),
  tl = Cons(hd = Succ(n = Z), tl = Nil)
)

In [26]:
def numListLen(l:NumList, acc:Int = 0) : Int = {
    l match {
        case Nil => acc
        case Cons(hd,tl) => numListLen(tl, acc+1)
    }
}

defined function numListLen

In [25]:
numListLen(l1)

res24: Int = 2

In [28]:
def concat(l1:NumList, l2:NumList) : NumList = {
    l1 match {
        case Nil => l2
        case Cons(hd,tl) =>  Cons(hd,concat(tl,l2))
    }
}

defined function concat

In [29]:
concat(l1,l1)

res28: NumList = Cons(
  hd = Succ(n = Succ(n = Z)),
  tl = Cons(
    hd = Succ(n = Z),
    tl = Cons(hd = Succ(n = Succ(n = Z)), tl = Cons(hd = Succ(n = Z), tl = Nil))
  )
)

In [30]:
def realConcat(l1:List[Int], l2:List[Int]) : List[Int] = {
    l1 match {
        case List() => l2
        case hd::tl =>  hd::(realConcat(tl,l2))
    }
}

defined function realConcat